# Chapter 4. Estimating Proportions
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap04.html)

A reminder of Bayes’s Theorem:

$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$

or

$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import CairoMakie as Cmk
import Distributions as Dsts

In [ ]:
include("pmf.jl")
import .ProbabilityMassFunction as Pmf

## The Euro Problem

In Information Theory, Inference, and Learning Algorithms, David MacKay poses this problem:

“A statistical statement appeared in The Guardian on Friday January 4, 2002:

> When spun on edge 250 times, a Belgian one-euro coin came up heads 140 times
> and tails 110. "It looks very suspicious to me", said Barry Blight, a statistics
> lecturer at the London School of Economics. "If the coin were unbiased, the
> chance of getting a result as extreme as that would be less than 7%."

“But [MacKay asks] do these data give evidence that the coin is biased rather than fair?”




## The Binomial Distribution

The probability that we get a total of $k$ heads is given by the [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution):

$\binom{n}{k}*p^{k}*(1-p)^{n-k}$

for any value of $k$ from 0 to $n$, including both.

The term $\binom{n}{k}$ is the binomial coefficient, usually pronounced “n choose k”.

We could evaluate the expression ourselves or use a library, like so

In [ ]:
n = 2
p = 0.5
k = 1

Dsts.pdf(Dsts.Binomial(n, p), k) |> x -> round(x, digits=3)

We can also use multiple values of $k$ with the functions

In [ ]:
ks = 0:1:n |> collect
ps = Dsts.pdf.(Dsts.Binomial(n, p))
ps = map(x -> round(x, digits=3), ps)
ps

We can put these probabilities in a `Pmf`

In [ ]:
pmfK = Pmf.Pmf(ks, ps)
pmfK

Here's what it looks like with `n=250` and `p=0.5`:

In [ ]:
pmfK = Pmf.getBinomialPmf(250, 0.5)

In [ ]:
fig = Pmf.drawLinesPriors(pmfK,
    "Binomial Distribution (n=250, p=0.5)",
    "Number of heads (k)",
    "PMF"
    )
fig

In [ ]:
Pmf.getNameMaxPrior(pmfK)

In [ ]:
# (125 + 1) because Julia's indexing starts at 1
pmfK.priors[126]

In MacKay's example, we got 140 heads, which is even less liekly than 125:

In [ ]:
# (140 + 1) because Julia's indexing starts at 1
pmfK.priors[141]

In [ ]:
Pmf.getTotalProbGEName(pmfK, "priors", 140)

The result is about 3.3%, which is less than the quoted 7%. The reason for the difference is that the statistician includes all outcomes “as extreme as” 140, which includes outcomes less than or equal to 110. (two tailed probability)

In [ ]:
Pmf.getTotalProbGEName(pmfK, "priors", 140) * 2

In [ ]:
# alternative solution (without Pmf)
Dsts.cdf(Dsts.Binomial(250, 0.5), 110) +
Dsts.ccdf(Dsts.Binomial(250, 0.5), 139)
# or just
# Dsts.cdf(Dsts.Binomial(250, 0.5), 110) * 2

## Bayesian Estimation

In [ ]:
# coins with different probs of getting heads
coins = Pmf.getPmfFromSeq(range(0, 1, 101) |> collect)

In [ ]:
likelihood_heads = copy(coins.names)
likelihood_tails = 1 .- likelihood_heads

likelihood_mapping = Dict(
   'h' => likelihood_heads,
   't' => likelihood_tails
)

In [ ]:
dataset = "h" ^ 140 * "t" ^ 110

In [ ]:
"""
    Update pmf with a given sequence of h and t
"""
function update_euro!(
    coins::Pmf.Pmf{T},
    dataset::String,
    prob_mapping::Dict{Char,Vector{Float64}}) where {T<:Union{Int,String,Float64}}

    coins.likelihoods .= 1
    for data in dataset
		coins.likelihoods .*= prob_mapping[data]
        # coins.likelihoods = coins.likelihoods ./ sum(coins.likelihoods)
	end
	Pmf.updatePosteriors!(coins, true)
    return nothing

end

In [ ]:
update_euro!(coins, dataset, likelihood_mapping)

In [ ]:
fig = Pmf.drawLinesPosteriors(coins,
    "Binomial Distribution (n=250, p=0.5),\n140/250 heads",
    "Number of heads (k)",
    "PMF"
    )
fig

In [ ]:
# index of coins with max priors
Pmf.getIndMaxPosterior(coins)

In [ ]:
# value for heads with max priors
Pmf.getNameMaxPosterior(coins)

## Triangle prior

Comparison between two priors:
- uniform
- triangle shaped

In [ ]:
uniform = Pmf.getPmfFromSeq(range(0, 1, 101) |> collect)

In [ ]:
shape = vcat(0:49, 50:-1:0)
shape = shape ./ sum(shape)
triangle = Pmf.Pmf(range(0, 1, 101) |> collect, shape)

In [ ]:
fig = Cmk.Figure(size=(600, 400))
Cmk.lines(fig[1, 1], uniform.names, uniform.priors,
    color="blue",
    axis=(;
        title="Uniform and triangle distributions",
        xlabel="Proportion of heads (x)",
        ylabel="Probability")
    )
Cmk.lines!(fig[1, 1], triangle.names, triangle.priors, color="orange")
fig

In [ ]:
update_euro!(uniform, dataset, likelihood_mapping);
update_euro!(triangle, dataset, likelihood_mapping);

In [ ]:
fig = Cmk.Figure(size=(600, 400))
Cmk.lines(fig[1, 1], uniform.names, uniform.posteriors,
    color="blue",
    axis=(;
        title="Uniform and triangle distributions",
        xlabel="Proportion of heads (x)",
        ylabel="Probability")
    )
Cmk.lines!(fig[1, 1], triangle.names, triangle.posteriors, color="orange")
fig

This is an example of **swamping the priors**: with enough data, people who start with different priors will tend to converge on the same posterior distribution.